In [1]:
import matplotlib.pyplot as plt
import math
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import folium
from collections import Counter
from ipynb.fs.defs.Local_search import get_row
from ipynb.fs.defs.Local_search import getName
from ipynb.fs.defs.functions import translate_name
from ipynb.fs.defs.functions import get_row
from ipynb.fs.defs.functions import get_column

df = pd.read_csv("../Dati/request_cr.csv", delimiter=",", encoding = "ISO-8859-1")
dataframe = pd.read_csv("../Dati/num_orders.csv", delimiter=";", encoding = "ISO-8859-1")
coor = pd.read_csv('../Dati/altro/CitiesLocation.csv', delimiter=";", encoding = "ISO-8859-1", index_col = 0)
dft = pd.read_csv("../Dati/distanze/corrected_times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur1_MVLS = pd.read_csv("../Dati/turni_f/multi_veich_LS1.csv", delimiter=";", encoding = "ISO-8859-1")
fur2_MVLS = pd.read_csv("../Dati/turni_f/multi_veich_LS2.csv", delimiter=";", encoding = "ISO-8859-1")
fur1_2opt = pd.read_csv("../Dati/turni_f/2-opt1_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur2_2opt = pd.read_csv("../Dati/turni_f/2-opt2_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur1_greedy = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";", encoding = "ISO-8859-1")
fur2_greedy = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";", encoding = "ISO-8859-1")

def manage_exception(string):
    if(string.lower().strip() == 'piadena drizzona'):
        return 'Piadena, Platina, 40'
    elif(string.lower().strip() == 'brescia'):
        return 'manerbio, insurrezione nazionale'
    elif(string.lower().strip() == 'centro fumetto a. pazienza' or string.lower().strip() == 'cremona. la piccola biblioteca' or string.lower().strip() == 'uff. bib. cremona'):
        return 'cremona, ugolani dati, 4'
    elif(string.lower().strip() == 'casale cremasco vidolasco'):
        return 'Casale Cremasco-Vidolasco, Roma, 56'
    elif(string.lower().strip() == 'cremona-m.i.t.o.'):
        return 'Cremona, Palosca n. 2'
    elif(string.lower().strip() == 'cremona - manin (scolastica)'):
        return 'Cremona, Felice Cavallotti, 2'
    elif(string.lower().strip() == 'gadesco pieve delmona'):
        return 'Gadesco-Pieve Delmona, Giovanni Lonati 9'
    elif(string.lower().strip() == 'cremona - cda'):
        return 'cremona, sesto 39'
    elif(string.lower().strip() == 'crema'):
        return 'Crema, Civerchi, 9'
    elif(string.lower().strip() == 'martignana di po, libertà, 64'):
        return 'martignana di po, libertà, 64'
    else:
        return string

In [54]:
loc = [coor.loc['Cremona, Ugolani Dati, 4']['Lat'], coor.loc['Cremona, Ugolani Dati, 4']['Lon']]
map_ = folium.Map(location = loc)
orig = ''
dest = ''


for i in range(len(dataframe)):
    try:
        orig = getName(coor, manage_exception(dataframe.iloc[i]['origin']))
        dest = getName(coor, manage_exception(dataframe.iloc[i]['destination']))
    except:
        continue
    point1 = [coor.loc[orig]['Lat'], coor.loc[orig]['Lon']]
    point2 = [coor.loc[dest]['Lat'], coor.loc[dest]['Lon']]
    points = [point1, point2]
    
    orders = dataframe.iloc[i]['num_orders']
    if(orders > 500):
        folium.PolyLine(points, color="black", weight=0.5, opacity=1).add_to(map_)
        #folium.PolyLine(points).add_to(map_)
    elif(orders >= 400 and orders < 500):
        folium.PolyLine(points, color="red", weight=0.5, opacity=1).add_to(map_)
    elif(orders >= 300 and orders < 400):
        folium.PolyLine(points, color="orange", weight=0.5, opacity=1).add_to(map_)
    elif(orders >= 200 and orders < 300):
        folium.PolyLine(points, color="darkblue", weight=0.5, opacity=1).add_to(map_)
    elif(orders >= 100 and orders < 200):
        folium.PolyLine(points, color="blue", weight=0.5, opacity=1).add_to(map_)
    elif(orders >= 0 and orders < 100):
        folium.PolyLine(points, color="lightblue", weight=0.5, opacity=1).add_to(map_)
    

C:\Users\Alberto\anaconda3\lib\site-packages\pandas\core\strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [13]:
#set(list(df['biblioteca_destinazione']))
origins = []
for x in set(list(df['biblioteca_partenza'])):
    try:
        origins.append(getName(dft, manage_exception(x)))
    except:
        continue
destins = []
for x in set(list(df['biblioteca_destinazione'])):
    try:
        destins.append(getName(dft, manage_exception(x)))
    except:
        continue
origins.sort()
destins.sort()

In [83]:
indx = [getName(dft, x) for x in dft.columns]
ord_numbers = pd.DataFrame({}, index = indx, columns = indx)
for i in range(len(dataframe)):
    try:
        orig = getName(dft, manage_exception(dataframe.iloc[i]['origin']))
        exc.append(dataframe.iloc[i]['origin'])
        dest = getName(dft, manage_exception(dataframe.iloc[i]['destination']))
        exc.append(dataframe.iloc[i]['destination'])
        orders = dataframe.iloc[i]['num_orders']
        ord_numbers.loc[orig,dest] = orders
    except:
        #print([orig, dest])
        continue  

In [84]:
ord_numbers=ord_numbers.replace(np.nan, 0)

In [85]:
ord_numbers.to_csv("../Dati/distanze/ord_numbers.csv", sep=";", encoding = "ISO-8859-1")

In [86]:
orders = pd.read_csv("../Dati/distanze/ord_numbers.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)


# Giorni di attesa

In [2]:
#Take schedulation and return a list that contains the day(in index) where the city appears
def getOccurenceDays(fur1, fur2, city, dframe, b):
    week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
    l = []
    for d in fur1.columns:
        if b == 1:
            cur_route1 = fur1[d].dropna()[1:len(fur1[d].dropna())-1]
        else:
            cur_route1 = fur1[d].dropna()
            
        if getName(dframe, city) in list(cur_route1):
            l.append(week.index(d))
            
    for d in fur2.columns:
        if b == 1:
            cur_route2 = fur2[d].dropna()[1:len(fur2[d].dropna())-1]
        else:
            cur_route2 = fur2[d].dropna()
            
        if getName(dframe, city) in list(cur_route2):
            l.append(week.index(d))
          
    return list(set(l))

def getDistance(s, e):
    if s == e:
        return 7
    if s < e:
        return e-s
    if s > e:
        return 7-(s-e)

def mean_wdays(fur1, fur2, city_s, city_e, dframe, b):
    s_list = getOccurenceDays(fur1, fur2, city_s, dframe, b)
    e_list = getOccurenceDays(fur1, fur2, city_e, dframe, b)
    results = []
    for s in s_list:
        for e in e_list:
            results.append(getDistance(s, e))
    
    if not results:
        #print(s_list, city_s, sep = ' $ ')
        #print(e_list, city_e, sep = ' $ ')
        return -1
    return round(sum(results) / len(results), 1)

In [3]:
def compute_mean_wdays(dframe, fur1, fur2, b):
    week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
    
    f1 = translate_name(dframe, fur1)
    f2 = translate_name(dframe, fur2)
    
    indx = [getName(dframe, x) for x in dframe.columns]
    waiting_days = pd.DataFrame({}, index = indx, columns = indx)
    
    for s in indx:
        for e in indx:
            waiting_days.loc[s, e] = mean_wdays(f1, f2, s, e, dframe, b)
            
    return waiting_days

In [44]:
compute_mean_wdays(dft, fur1_MVLS, fur2_MVLS,1 ).to_csv("../Dati/distanze/waiting_days_MVLS.csv", sep=";", encoding = "ISO-8859-1")
compute_mean_wdays(dft, fur1_2opt, fur2_2opt, 1).to_csv("../Dati/distanze/waiting_days_2opt.csv", sep=";", encoding = "ISO-8859-1")
compute_mean_wdays(dft, fur1_greedy, fur2_greedy, 0).to_csv("../Dati/distanze/waiting_days_greedy.csv", sep=";", encoding = "ISO-8859-1")

In [4]:
waiting_days_MVLS = pd.read_csv("../Dati/distanze/waiting_days_MVLS.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
waiting_days_2opt = pd.read_csv("../Dati/distanze/waiting_days_2opt.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
waiting_days_greedy = pd.read_csv("../Dati/distanze/waiting_days_greedy.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
orders = pd.read_csv("../Dati/distanze/ord_numbers.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
indx = [getName(dft, x) for x in dft.columns]
weighted_index_MVLS = pd.DataFrame({}, index = indx, columns = indx)
weighted_index_2opt = pd.DataFrame({}, index = indx, columns = indx)
weighted_index_greedy = pd.DataFrame({}, index = indx, columns = indx)

for s in indx:
    for e in indx:
        weighted_index_MVLS.loc[s, e] = waiting_days_MVLS.loc[s, e] * orders.loc[s, e]
        weighted_index_2opt.loc[s, e] = waiting_days_2opt.loc[s, e] * orders.loc[s, e]
        weighted_index_greedy.loc[s, e] = waiting_days_greedy.loc[s, e] * orders.loc[s, e]

In [6]:
weighted_index_MVLS.to_csv("../Dati/distanze/weighted_wmean_MVLS.csv", sep=";", encoding = "ISO-8859-1")
weighted_index_2opt.to_csv("../Dati/distanze/weighted_wmean_2opt.csv", sep=";", encoding = "ISO-8859-1")
weighted_index_greedy.to_csv("../Dati/distanze/weighted_wmean_greedy.csv", sep=";", encoding = "ISO-8859-1")

In [4]:
def get_carla(fur1, fur2, city_s, city_e, dframe, b):
    s_list = getOccurenceDays(fur1, fur2, city_s, dframe, b)
    e_list = getOccurenceDays(fur1, fur2, city_e, dframe, b)
    results = []
    dists = []
    ocs = []
    for s in s_list:
        for e in e_list:
            results.append((s, e))
            dists.append(getDistance(s, e))
    
    if not results:
        #print(s_list, city_s, sep = ' $ ')
        #print(e_list, city_e, sep = ' $ ')
        return -1
    return results, dists

indx = [getName(dft, x) for x in dft.columns]

In [5]:
dat = pd.DataFrame({}, columns = ['start', 'start_day', 'destination', 'destination_day', 'product'])
orders = pd.read_csv("../Dati/distanze/ord_numbers.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
f1 = translate_name(dft, fur1_2opt)
f2 = translate_name(dft, fur2_2opt)

In [12]:


"""for i in indx:
    for j in indx:
        l = get_carla(f1, f1, i, j, dft, 1)
        if l != -1:
            for x in l:
                dat = dat.append({'start' : indx.index(i),  'start_day': x[0], 'destination': indx.index(j), 'destination_day': x[1], 'product': round(x[2]*orders.loc[i, j], 1)}, ignore_index = True)
        else:
             dat = dat.append({'start' : indx.index(i),  'start_day': x[0], 'destination': indx.index(j), 'destination_day': x[1], 'product': round(x[2]*orders.loc[i, j], 1)}, ignore_index = True)
"""

"for i in indx:\n    for j in indx:\n        l = get_carla(f1, f1, i, j, dft, 1)\n        if l != -1:\n            for x in l:\n                dat = dat.append({'start' : indx.index(i),  'start_day': x[0], 'destination': indx.index(j), 'destination_day': x[1], 'product': round(x[2]*orders.loc[i, j], 1)}, ignore_index = True)\n        else:\n             dat = dat.append({'start' : indx.index(i),  'start_day': x[0], 'destination': indx.index(j), 'destination_day': x[1], 'product': round(x[2]*orders.loc[i, j], 1)}, ignore_index = True)\n"

In [5]:
from tqdm.notebook import tqdm as tqdm

In [6]:
dat = pd.DataFrame({}, columns = ['start', 'start_day', 'destination', 'destination_day', 'product'])
orders = pd.read_csv("../Dati/distanze/ord_numbers.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
f1 = translate_name(dft, fur1_greedy)
f2 = translate_name(dft, fur2_greedy)
k = indx
k.pop(15)
week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
indx = [getName(dft, x) for x in dft.columns]
indx.pop(15)
for i in tqdm(range(len(k))):
    f1_occ = getOccurenceDays(f1, f2, indx[i], dft, 1)
    for d1 in week:
        if week.index(d1) in f1_occ:
            for j in k:
                f2_occ = getOccurenceDays(f1, f2, j, dft, 1)
                for d2 in week:
                    if indx[i] == j:
                        dat = dat.append({'start' : i,  'start_day': week.index(d1), 'destination': indx.index(j), 'destination_day': week.index(d2), 
                                          'product': 0}, ignore_index = True)
                        continue
                        
                    if week.index(d2) in f2_occ:
                        l, dists = get_carla(f1, f2, indx[i], j, dft, 1)
                        dat = dat.append({'start' : i,  'start_day': week.index(d1), 'destination': indx.index(j), 'destination_day': week.index(d2), 
                                          'product': round(getDistance(week.index(d1), week.index(d2))*orders.loc[indx[i], j], 
                                                           1)}, ignore_index = True)
                    else:
                        dat = dat.append({'start' : i,  'start_day': week.index(d1), 'destination': indx.index(j), 'destination_day': week.index(d2), 
                                          'product': 0}, ignore_index = True)
        else:
            for w in k:
                for d3 in week:
                    dat = dat.append({'start' : i,  'start_day': week.index(d1), 'destination': indx.index(w), 'destination_day': week.index(d3), 
                                      'product': 0}, ignore_index = True)

In [8]:
#pd.set_option('display.float_format', '{:.2f}'.format)
dat.to_csv("../Dati/distanze/datoss.dat", sep=" ", encoding = "ISO-8859-1", index = False, header = False)
dfprova = pd.read_csv("../Dati/distanze/dat.dat", delimiter=" ", encoding = "ISO-8859-1", names = ['x', 'y', 'z', 'w', 'v'])
len(dfprova)

105625

## Interpretation of biblio.sol

In [2]:
def parseEl(x):
    temp = x[x.find("(")+1:x.find(")")]
    temp = temp.split(",")
    return temp[0], temp[1]

indx = [getName(dft, x) for x in dft.columns]
week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
biblio = pd.read_csv("../Dati/turni_f/optimized_start/biblio_oss.sol", delimiter=" ", encoding = "ISO-8859-1", names = ['x','b'])
x = pd.DataFrame({}, columns=["city", "day"])
biblio = biblio[biblio["b"]==1]
for el in biblio["x"]:
    s, d = parseEl(el)
    if(int(s)>=15):
        x = x.append({"city" : indx[int(s)+1], "day":week[int(d)]}, ignore_index = True)
    else:
        x = x.append({"city" : indx[int(s)], "day":week[int(d)]}, ignore_index = True)

In [4]:
lenght = max([len(x[x["day"]==d]) for d in week])
optimized_start_schedule = pd.DataFrame({})
optimized_start_schedule["pad"] = pd.Series([np.nan]*lenght)

for d in week:
    selection = x["city"][x["day"]==d].reset_index()["city"]
    optimized_start_schedule = optimized_start_schedule.join(
                                        pd.DataFrame(x["city"][x["day"]==d].reset_index()["city"].rename(d)))
optimized_start_schedule = optimized_start_schedule[week]
optimized_start_schedule

,lunedi,martedi,mercoledi,giovedi,venerdi
0,"Agnadello, Marconi, 6","Capergnanica, XI febbraio, 7","Bonemerse, Roma, 25","Capralba, Piave, 2","Bagnolo Cremasco, Roma, 3"
1,"Annicco, Giuseppe Garibaldi, 6","Casalbuttano ed Uniti, Jacini, 23","Casale Cremasco-Vidolasco, Roma, 56","Crema, Civerchi, 9","Casalmaggiore, Guglielmo Marconi, 8"
2,"Camisano, Trieste, 57","Casalmaggiore, Guglielmo Marconi, 8","Crema, Civerchi, 9","Cremona, Ugolani Dati, 4","Casalpusterlengo, Pusterla 7"
3,"Castelleone, Roma, 67","Castelleone, Roma, 67","Fiesco, Giuseppe Parini, 10","Pandino, Castello, 15","Castelverde, Ubaldo Ferrari, 15"
4,"Cingia de' Botti, Giuseppina, 79","Crema, Civerchi, 9","Grontardo, Roma, 16","Persico Dosimo, Albert ,7","Crema, Civerchi, 9"
5,"Cremona, Ugolani Dati, 4","Gadesco-Pieve Delmona, Giovanni Lonati 9","Gussola, Comaschi, 1","Pescarolo ed Uniti, Giuseppe Mazzini, 73","Cremona, Ugolani Dati, 4"
6,"Dovera, Oldrini 2","Grumello Cremonese ed Uniti, Roma, 2","Isola Dovarese, Giacomo Matteotti, 1","Piadena, Platina, 40","Izano, Gerardo da Izano, 4/C"
7,"Gadesco-Pieve Delmona, Giovanni Lonati 9","Montodine, XXV Aprile, 10","Paderno Ponchielli, Municipio, 3","Rivolta d'Adda, Stefano Renzi, 5","Ostiano, Mazzini, 1"
8,"Madignano, Portici, 4","Offanengo, Clavelli Martini, 5","Pandino, Castello, 15","Romanengo, Guaiarini, 14","Piadena, Platina, 40"
9,"Martignana di Po, Libertà, 64","Ostiano, Mazzini, 1","Pescarolo ed Uniti, Giuseppe Mazzini, 73","San Giovanni in Croce, Feudatari, 25 A","Pianengo, Nenni"


In [5]:
optimized_start_schedule.to_csv("../Dati/turni_f/optimized_start/optimized_start_schedule.csv", sep=";", encoding = "ISO-8859-1", index = False)

## Extract f1 and f2 optimized start schedule

In [6]:
oss = pd.read_csv("../Dati/turni_f/optimized_start/optimized_start_schedule.csv", delimiter=";", encoding = "ISO-8859-1")
oss_f1 = pd.DataFrame({})
oss_f1["pad"] = pd.Series([np.nan]*len(oss))
oss_f2 = pd.DataFrame({})
oss_f2["pad"] = pd.Series([np.nan]*len(oss))
week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]

for day in week:
    if day in ["lunedi", "venerdi"]:
        oss_f1 = oss_f1.join(oss[day])
    else:
        start = dft[get_row(dft, 'Cremona, sesto 39').values]
        f1 = []
        f2 = []
        row1 = start
        row2 = start
        for to_insert in oss[day].dropna():
            value1=row1.loc[:,get_column(dft, to_insert).values]
            value2=row2.loc[:,get_column(dft, to_insert).values]
            if(float(value1.values[0])<=float(value2.values[0])):
                f1.append(to_insert)
                row1 = dft[get_row(dft, to_insert).values]
            else:
                f2.append(to_insert)
                row2 = dft[get_row(dft, to_insert).values]
                           
        oss_f1 = oss_f1.join(pd.Series(f1, dtype = str).rename(day))
        oss_f2 = oss_f2.join(pd.Series(f2, dtype = str).rename(day))

oss_f1 = oss_f1[week]
oss_f2 = oss_f2[["martedi", "mercoledi", "giovedi"]]

In [7]:
oss_f1.to_csv("../Dati/turni_f/optimized_start/oss_f1.csv", sep=";", encoding = "ISO-8859-1", index = False)
oss_f2.to_csv("../Dati/turni_f/optimized_start/oss_f2.csv", sep=";", encoding = "ISO-8859-1", index = False)